# 第2章 機械学習を使った分析を行ってみよう
ここでは、機械学習の基礎を学ぶうえで必要なプログラムを実行していく流れを学んでいきます。  

In [ ]:
#Colaboratory環境の設定
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/MathProgramming/Chapter2

In [ ]:
#ライブラリの設定
!pip install -q -r ./requirements.txt

## 2-1 顧客の行動パターンの類似度を計算しよう

### データ読み込み

In [ ]:
import pandas as pd
df_info = pd.read_csv("accomodation_info.csv", index_col=0, parse_dates=[0])
df_info

### 特徴ベクトル可視化（特徴ベクトルとして利用回数の時系列データを利用）

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
# indexの抽出
x_0 = df_info.resample('M').count()
x_0 = x_0.drop(x_0.columns.values,axis=1)
# 順位の設定
i_rank = 1
j_rank = 2
# 顧客IDの抽出
i_id = df_info['顧客ID'].value_counts().index[i_rank]
j_id = df_info['顧客ID'].value_counts().index[j_rank]
# 月ごとの利用回数を特徴量として抽出
x_i = df_info[df_info['顧客ID']==i_id].resample('M').count()
x_j = df_info[df_info['顧客ID']==j_id].resample('M').count()
# 欠損値があった場合の穴埋め
x_i = pd.concat([x_0, x_i], axis=1).fillna(0)
x_j = pd.concat([x_0, x_j], axis=1).fillna(0)
# 描画
plt.plot(x_i)
plt.plot(x_j)
plt.xticks(rotation=60)
plt.show()

### 類似度計算

In [ ]:
import pandas as pd
import numpy as np
# 特徴ベクトルの差を計算
dx = x_i.iloc[:,0].values-x_j.iloc[:,0].values
# ベクトルノルム（距離）を計算
n = np.linalg.norm(dx)
# 次元による正規化
num_dim = len(x_i)
d = n/num_dim
print("類似度:",d)

## 2-3 大口顧客の類似性を主成分分析によって確認しよう

### 特徴ベクトル抽出

In [ ]:
import pandas as pd
# indexの調整
x_0 = df_info.resample('M').count()
x_0 = x_0.drop(x_0.columns.values,axis=1)
# 配列を準備
list_vector = []
# 人数の設定
num = 100
for i_rank in range(num):
    # 顧客IDの抽出
    i_id = df_info['顧客ID'].value_counts().index[i_rank]
    # 月ごとの利用回数を特徴量として抽出
    x_i = df_info[df_info['顧客ID']==i_id].resample('M').count()
    # 欠損値があった場合の穴埋め
    x_i = pd.concat([x_0, x_i], axis=1).fillna(0)
    # 特徴ベクトルとして追加
    list_vector.append(x_i.iloc[:,0].values.tolist())

### 主成分分析(PCA)による可視化

In [ ]:
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt
# 特徴ベクトルを変換
features = np.array(list_vector)
# 主成分分析を実施
pca = PCA()
pca.fit(features)
# 特徴ベクトルを主成分に変換
transformed = pca.fit_transform(features)
# 可視化
for i in range(len(transformed)):
    plt.scatter(transformed[i,0],transformed[i,1],color="k")
    plt.text(transformed[i,0],transformed[i,1],str(i))
plt.show()

## 2-4. 大口顧客の行動パターンを時系列によって確かめよう

In [ ]:
import pandas as pd
# indexの抽出
x_0 = df_info.resample('M').count()
x_0 = x_0.drop(x_0.columns.values,axis=1)

# 順位の設定
list_rank = [0,1,2]
x = []
for i_rank in list_rank:
    # 顧客IDの抽出
    i_id = df_info['顧客ID'].value_counts().index[i_rank]
    # 月ごとの利用回数を特徴量として抽出
    x_i = df_info[df_info['顧客ID']==i_id].resample('M').count()
    # 欠損値があった場合の穴埋め
    x_i = pd.concat([x_0, x_i], axis=1).fillna(0)
    # 描画
    plt.plot(x_i)
    plt.xticks(rotation=60)
plt.show()

## 2-5. 大口顧客同士の行動パターンの違いをクラスタリングによって可視化しよう

### クラスタリング(k-means法)による分類

In [ ]:
from sklearn.cluster import KMeans
# クラスター数を設定
num_of_cluster = 4
# クラスターに分類
model = KMeans(n_clusters=num_of_cluster, random_state=0)
model.fit(features)
pred_class = model.labels_
print(pred_class)

### 主成分分析(PCA)による可視化

In [ ]:
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt

# 主成分分析を実施
pca = PCA()
pca.fit(features)
# 特徴ベクトルを主成分に変換
transformed = pca.fit_transform(features)
# 可視化
plt.figure(figsize=(12, 8))
plt.scatter(transformed[:,0],transformed[:,1],c=pred_class)
for i in range(len(transformed)):
    text = str(i) + "(" + str(pred_class[i]) + ")"
    plt.text(transformed[i,0],transformed[i,1],text)
plt.show()

## 2-6. 決定木によって行動の原因を推定してみよう

### 目的変数の設定

In [ ]:
import numpy as np
# 分析したいクラスを設定する
target_class = 1
# 目的変数を作成する
num = len(pred_class)
data_o = np.zeros(num)
for i in range(num):
    if pred_class[i]==target_class:
        data_o[i] = True
    else:
        data_o[i] = False
print(data_o)

### 説明変数の設定

In [ ]:
# 説明変数を作成する
data_e = features
print(data_e)

### モデル構築

In [ ]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz
# 決定木のモデル構築を実行する
clf = DecisionTreeClassifier(max_depth=2)
clf = clf.fit(data_e, data_o)

### 結果の描画

In [ ]:
from dtreeviz.trees import dtreeviz

# indexの抽出
x_0 = df_info.resample('M').count()
x_0 = x_0.drop(x_0.columns.values,axis=1)
time_index = x_0.index
print(time_index)

# 決定木を描画
viz = dtreeviz(
    clf,
    data_e, 
    data_o,
    target_name='Class',
    feature_names=time_index,
    class_names=['False','True'],
) 
viz

## 2-7. 決定木の分類結果を可視化し、分類精度を評価しよう

### 分類結果の可視化

In [ ]:
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as pat

# 分類を実施
pred_tree = clf.predict(data_e)

# 主成分分析を実施
pca = PCA()
pca.fit(features)
# 特徴ベクトルを主成分に変換
transformed = pca.fit_transform(features)
# 可視化
plt.figure(figsize=(12, 8))
plt.scatter(transformed[:,0],transformed[:,1],c=pred_class)
for i in range(len(transformed)):
    if pred_tree[i]==1:
        if pred_class[i]==1:
            temp_color = "k"
            temp_lw = 1.0
        else:
            temp_color = "b"
            temp_lw = 3.0
        circle = pat.Circle(xy=(transformed[i,0],transformed[i,1]), radius=1.0, ec=temp_color ,fill=False, linewidth = temp_lw)
        plt.axes().add_artist(circle)
    else:
        if pred_class[i]==1:
            temp_color = "r"
            temp_lw = 3.0
            circle = pat.Circle(xy=(transformed[i,0],transformed[i,1]), radius=1.0, ec=temp_color ,fill=False, linewidth = temp_lw)
            plt.axes().add_artist(circle)
    text = str(i) + "(" + str(pred_class[i]) + ")"
    plt.text(transformed[i,0],transformed[i,1],text)
plt.show()
%matplotlib inline

### 混同行列の出力

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(data_o, pred_tree)
print(cm)

## 2-8. 予測の精度を評価する流れを理解しよう

### データセットを訓練データとテストデータに分割

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(features,data_o)

### 訓練データによるモデル構築

In [ ]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz
clf = DecisionTreeClassifier(max_depth=2)
clf = clf.fit(x_train, y_train)

### テストデータによる評価

In [ ]:
from sklearn.metrics import confusion_matrix

# スコア計算
score = clf.score(x_test, y_test)
print("スコア:",score)

# 混同行列生成
pred_tree = clf.predict(x_test)
cm = confusion_matrix(y_test, pred_tree)
print("混同行列")
print(cm)

## 2-9. さまざまな分類アルゴリズムを比較しよう

### ランダムフォレストとの比較

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

# データセットを訓練データとテストデータに分割
x_train, x_test, y_train, y_test = train_test_split(features,data_o)

# 訓練データによるモデル構築
model = RandomForestClassifier(bootstrap=True, n_estimators=10, max_depth=None, random_state=1)
clf = model.fit(x_train, y_train)

# テストデータによる評価
# スコア計算
score = clf.score(x_test, y_test)
print("スコア:",score)

# 混同行列生成
pred_tree = clf.predict(x_test)
cm = confusion_matrix(y_test, pred_tree)
print("混同行列")
print(cm)

### SVMとの比較

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

# データセットを訓練データとテストデータに分割
x_train, x_test, y_train, y_test = train_test_split(features,data_o)

# 訓練データによるモデル構築
model = SVC(kernel='rbf')
clf = model.fit(x_train, y_train)

# テストデータによる評価
# スコア計算
score = clf.score(x_test, y_test)
print("スコア:",score)

# 混同行列生成
pred_tree = clf.predict(x_test)
cm = confusion_matrix(y_test, pred_tree)
print("混同行列")
print(cm)

## 2-10. サポートベクトル回帰によって時系列予測をしてみよう

In [ ]:
from sklearn import svm
from sklearn.model_selection import train_test_split

# データを作成
data_target = data_e[data_o==1]
data_y = data_target
data_x = np.stack([np.arange(0,len(data_target[0])) for _ in range(len(data_target))], axis=0)
data_y = np.ravel(data_y)
data_x = np.ravel(data_x)

# データセットを訓練データとテストデータに分割
x_train, x_test, y_train, y_test = train_test_split(data_x,data_y)

# 訓練データによるモデル構築（サポートベクトル回帰）
model = svm.SVR(kernel='rbf', C=1)
reg = model.fit(x_train.reshape(-1, 1),y_train.reshape(-1, 1))

# 予測曲線を描画
x_pred = np.arange(len(data_target[0])).reshape(-1, 1)
y_pred = model.predict(x_pred)
plt.plot(data_x,data_y,"k.")
plt.plot(x_pred,y_pred,"r.-")
plt.show()

# 決定係数R^2
reg.score(x_test.reshape(-1, 1),y_test.reshape(-1, 1))